# Amazon SageMaker Model Monitor
This notebook shows how to:
* Host a machine learning model in Amazon SageMaker and capture inference requests, results, and metadata 
* Analyze a training dataset to generate baseline constraints
* Monitor a live endpoint for violations against constraints

---
## Background

Amazon SageMaker provides every developer and data scientist with the ability to build, train, and deploy machine learning models quickly. Amazon SageMaker is a fully-managed service that encompasses the entire machine learning workflow. You can label and prepare your data, choose an algorithm, train a model, and then tune and optimize it for deployment. You can deploy your models to production with Amazon SageMaker to make predictions and lower costs than was previously possible.

In addition, Amazon SageMaker enables you to capture the input, output and metadata for invocations of the models that you deploy. It also enables you to analyze the data and monitor its quality. In this notebook, you learn how Amazon SageMaker enables these capabilities.

---
## Setup

To get started, make sure you have these prerequisites completed.

* Specify an AWS Region to host your model.
* An IAM role ARN exists that is used to give Amazon SageMaker access to your data in Amazon Simple Storage Service (Amazon S3). See the documentation for how to fine tune the permissions needed. 
* Create an S3 bucket used to store the data used to train your model, any additional model data, and the data captured from model invocations. For demonstration purposes, you are using the same bucket for these. In reality, you might want to separate them with different security policies.

In [1]:
import boto3
import os
import sagemaker
from sagemaker import get_execution_role

region = boto3.Session().region_name
role = get_execution_role()
sess = sagemaker.session.Session()
bucket = sess.default_bucket() 
prefix = 'tf-2-workflow'

s3_capture_upload_path = 's3://{}/{}/monitoring/datacapture'.format(bucket, prefix)

reports_prefix = '{}/reports'.format(prefix)
s3_report_path = 's3://{}/{}'.format(bucket,reports_prefix)

print("Capture path: {}".format(s3_capture_upload_path))
print("Report path: {}".format(s3_report_path))

Capture path: s3://sagemaker-us-east-1-062083580489/tf-2-workflow/monitoring/datacapture
Report path: s3://sagemaker-us-east-1-062083580489/tf-2-workflow/reports


# PART A: Capturing real-time inference data from Amazon SageMaker endpoints
Create an endpoint to showcase the data capture capability in action.


### Deploy the model to Amazon SageMaker
Start with deploying the trained TensorFlow model from lab 03.

In [2]:
import boto3

def get_latest_training_job_name(base_job_name):
    client = boto3.client('sagemaker')
    response = client.list_training_jobs(NameContains=base_job_name, SortBy='CreationTime', 
                                         SortOrder='Descending', StatusEquals='Completed')
    if len(response['TrainingJobSummaries']) > 0 :
        return response['TrainingJobSummaries'][0]['TrainingJobName']
    else:
        raise Exception('Training job not found.')

def get_training_job_s3_model_artifacts(job_name):
    client = boto3.client('sagemaker')
    response = client.describe_training_job(TrainingJobName=job_name)
    s3_model_artifacts = response['ModelArtifacts']['S3ModelArtifacts']
    return s3_model_artifacts

latest_training_job_name = get_latest_training_job_name('tf-2-workflow')
print(latest_training_job_name)
model_path = get_training_job_s3_model_artifacts(latest_training_job_name)
print(model_path)

tf-2-workflow-2021-03-13-20-59-58-959
s3://sagemaker-us-east-1-062083580489/tf-2-workflow-2021-03-13-20-59-58-959/output/model.tar.gz


Here, you create the model object with the image and model data.

In [3]:
from sagemaker.tensorflow.model import TensorFlowModel

tensorflow_model = TensorFlowModel(model_data = model_path,
                                  role = role,
                                  framework_version = '2.3.1')


In [4]:
from time import gmtime, strftime
from sagemaker.model_monitor import DataCaptureConfig

endpoint_name = 'tf-2-workflow-endpoint-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)

predictor = tensorflow_model.deploy(initial_instance_count=1,
                            instance_type='ml.m5.xlarge',
                            endpoint_name=endpoint_name,
                            data_capture_config=DataCaptureConfig(
                                enable_capture=True,
                                sampling_percentage=100,
                                destination_s3_uri=s3_capture_upload_path))

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


tf-2-workflow-endpoint-2021-03-14-14-29-48
-------------!

In [5]:
import numpy as np
from tensorflow.python.keras.datasets import boston_housing
from sklearn.preprocessing import StandardScaler

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

## Invoke the deployed model

You can now send data to this endpoint to get inferences in real time. Because you enabled the data capture in the previous steps, the request and response payload, along with some additional metadata, is saved in the Amazon Simple Storage Service (Amazon S3) location you have specified in the DataCaptureConfig.

This step invokes the endpoint with included sample data for about 3 minutes. Data is captured based on the sampling percentage specified and the capture continues until the data capture option is turned off.

In [6]:
%%time 

import time

print("Sending test traffic to the endpoint {}. \nPlease wait...".format(endpoint_name))

flat_list =[]
for item in x_test:
    result = predictor.predict(item)['predictions'] 
    flat_list.append(float('%.1f'%(np.array(result))))
    time.sleep(1.8)
    
print("Done!")
print('predictions: \t{}'.format(np.array(flat_list)))

Sending test traffic to the endpoint tf-2-workflow-endpoint-2021-03-14-14-29-48. 
Please wait...
Done!
predictions: 	[27.2 28.5 33.5 22.6 33.5 28.6 33.5 33.5 33.5 27.6 27.2 28.6 22.6 33.5
 27.2 33.5 33.5 28.6 27.6 27.6 27.2 27.2 33.5 28.6 33.5 27.6 33.5 33.5
 27.2 33.5 28.6 27.2 33.5 33.5 27.6 27.6 22.6 27.6 27.6 33.5 33.5 33.5
 27.6 33.5 33.5 33.5 33.5 33.5 27.6 33.5 33.5 33.5 27.6 28.6 28.7 33.5
 27.6 33.5 33.5 33.5 27.6 27.6 27.2 28.6 33.5 33.5 27.2 33.5 27.2 27.6
 27.6 33.5 27.6 27.6 33.5 33.5 33.5 33.5 33.5 27.6 33.5 33.5 28.6 27.6
 33.5 28.6 27.6 33.5 33.5 27.6 33.5 28.6 33.5 33.5 33.5 22.6 33.5 27.6
 33.5 28.6 33.5 27.6]
CPU times: user 295 ms, sys: 24 ms, total: 319 ms
Wall time: 3min 5s


## View captured data

Now list the data capture files stored in Amazon S3. You should expect to see different files from different time periods organized based on the hour in which the invocation occurred. The format of the Amazon S3 path is:

`s3://{destination-bucket-prefix}/{endpoint-name}/{variant-name}/yyyy/mm/dd/hh/filename.jsonl`

<b>Note that the delivery of capture data to Amazon S3 can require a couple of minutes so next cell might error. If this happens, please retry after a minute.</b>

In [7]:
s3_client = boto3.Session().client('s3')
result = s3_client.list_objects(Bucket=bucket, Prefix='tf-2-workflow/monitoring/datacapture/')
capture_files = [capture_file.get("Key") for capture_file in result.get('Contents')]
print("Found Capture Files:")
print("\n ".join(capture_files))

Found Capture Files:
tf-2-workflow/monitoring/datacapture/tf-2-workflow-endpoint-2021-03-14-14-29-48/AllTraffic/2021/03/14/14/36-22-159-664a6a05-631d-4117-b20e-e8dc51b13faf.jsonl
 tf-2-workflow/monitoring/datacapture/tf-2-workflow-endpoint-2021-03-14-14-29-48/AllTraffic/2021/03/14/14/37-22-285-11611277-c98a-4c80-8a07-01180be29c8f.jsonl


Next, view the contents of a single capture file. Here you should see all the data captured in an Amazon SageMaker specific JSON-line formatted file. Take a quick peek at the first few lines in the captured file.

In [8]:
def get_obj_body(obj_key):
    return s3_client.get_object(Bucket=bucket, Key=obj_key).get('Body').read().decode("utf-8")

capture_file = get_obj_body(capture_files[-1])
print(capture_file[:2000])

{"captureData":{"endpointInput":{"observedContentType":"application/json","mode":"INPUT","data":"[0.19539, 0.0, 10.81, 0.0, 0.413, 6.245, 6.2, 5.2873, 4.0, 305.0, 19.2, 377.17, 7.54]","encoding":"JSON"},"endpointOutput":{"observedContentType":"application/json","mode":"OUTPUT","data":"{\n    \"predictions\": [[33.4771194]\n    ]\n}","encoding":"JSON"}},"eventMetadata":{"eventId":"30971f7a-2baa-4e82-ba6c-8c680acff47c","inferenceTime":"2021-03-14T14:37:22Z"},"eventVersion":"0"}
{"captureData":{"endpointInput":{"observedContentType":"application/json","mode":"INPUT","data":"[13.0751, 0.0, 18.1, 0.0, 0.58, 5.713, 56.7, 2.8237, 24.0, 666.0, 20.2, 396.9, 14.76]","encoding":"JSON"},"endpointOutput":{"observedContentType":"application/json","mode":"OUTPUT","data":"{\n    \"predictions\": [[27.6045589]\n    ]\n}","encoding":"JSON"}},"eventMetadata":{"eventId":"a03cc388-1478-419f-82d1-8236f9c95650","inferenceTime":"2021-03-14T14:37:24Z"},"eventVersion":"0"}
{"captureData":{"endpointInput":{"obse

Finally, the contents of a single line is present below in a formatted JSON file so that you can observe a little better.

In [9]:
import json
print(json.dumps(json.loads(capture_file.split('\n')[0]), indent=2))

{
  "captureData": {
    "endpointInput": {
      "observedContentType": "application/json",
      "mode": "INPUT",
      "data": "[0.19539, 0.0, 10.81, 0.0, 0.413, 6.245, 6.2, 5.2873, 4.0, 305.0, 19.2, 377.17, 7.54]",
      "encoding": "JSON"
    },
    "endpointOutput": {
      "observedContentType": "application/json",
      "mode": "OUTPUT",
      "data": "{\n    \"predictions\": [[33.4771194]\n    ]\n}",
      "encoding": "JSON"
    }
  },
  "eventMetadata": {
    "eventId": "30971f7a-2baa-4e82-ba6c-8c680acff47c",
    "inferenceTime": "2021-03-14T14:37:22Z"
  },
  "eventVersion": "0"
}


As you can see, each inference request is captured in one line in the jsonl file. The line contains both the input and output merged together. In the example, you provided the ContentType as `text/csv` which is reflected in the `observedContentType` value. Also, you expose the encoding that you used to encode the input and output payloads in the capture format with the `encoding` value.

To recap, you observed how you can enable capturing the input or output payloads to an endpoint with a new parameter. You have also observed what the captured format looks like in Amazon S3. Next, continue to explore how Amazon SageMaker helps with monitoring the data collected in Amazon S3.

# PART B: Model Monitor - Baselining and continuous monitoring

In addition to collecting the data, Amazon SageMaker provides the capability for you to monitor and evaluate the data observed by the endpoints. For this:
1. Create a baseline with which you compare the realtime traffic. 
1. Once a baseline is ready, setup a schedule to continously evaluate and compare against the baseline.

## 1. Constraint suggestion with baseline/training dataset

The training dataset with which you trained the model is usually a good baseline dataset. Note that the training dataset data schema and the inference dataset schema should exactly match (i.e. the number and order of the features).

From the training dataset you can ask Amazon SageMaker to suggest a set of baseline `constraints` and generate descriptive `statistics` to explore the data. For this example, upload the training dataset that was used to train the pre-trained model included in this example. If you already have it in Amazon S3, you can directly point to it.

### Prepare training dataset with headers

In [10]:
import pandas as pd
dt = pd.DataFrame(data = x_train, 
                  columns = ["CRIM", "ZN", "INDUS", "CHAS","NOX","RM","AGE","DIS","RAD","TAX","PTRATIO","B","LSTAT"])

dt.to_csv("training-dataset-with-header.csv", index = False)

In [11]:
# copy over the training dataset to Amazon S3 (if you already have it in Amazon S3, you could reuse it)
baseline_prefix = prefix + '/baselining'
baseline_data_prefix = baseline_prefix + '/data'
baseline_results_prefix = baseline_prefix + '/results'

baseline_data_uri = 's3://{}/{}'.format(bucket,baseline_data_prefix)
baseline_results_uri = 's3://{}/{}'.format(bucket, baseline_results_prefix)
print('Baseline data uri: {}'.format(baseline_data_uri))
print('Baseline results uri: {}'.format(baseline_results_uri))


Baseline data uri: s3://sagemaker-us-east-1-062083580489/tf-2-workflow/baselining/data
Baseline results uri: s3://sagemaker-us-east-1-062083580489/tf-2-workflow/baselining/results


In [12]:
training_data_file = open("training-dataset-with-header.csv", 'rb')
s3_key = os.path.join(baseline_prefix, 'data', 'training-dataset-with-header.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(s3_key).upload_fileobj(training_data_file)

### Create a baselining job with training dataset

Now that you have the training data ready in Amazon S3, start a job to `suggest` constraints. `DefaultModelMonitor.suggest_baseline(..)` starts a `ProcessingJob` using an Amazon SageMaker provided Model Monitor container to generate the constraints.

In [13]:
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat

my_default_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
)

my_default_monitor.suggest_baseline(
    baseline_dataset=baseline_data_uri+'/training-dataset-with-header.csv',
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_results_uri,
    wait=True
)


Job Name:  baseline-suggestion-job-2021-03-14-14-45-14-849
Inputs:  [{'InputName': 'baseline_dataset_input', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-062083580489/tf-2-workflow/baselining/data/training-dataset-with-header.csv', 'LocalPath': '/opt/ml/processing/input/baseline_dataset_input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'monitoring_output', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-062083580489/tf-2-workflow/baselining/results', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
........................2021-03-14 14:49:03,145 - __main__ - INFO - All params:{'ProcessingJobArn': 'arn:aws:sagemaker:us-east-1:062083580489:processing-job/baseline-suggestion-job-2021-03-14-14-45-14-849', 'ProcessingJobName': 'baseline-suggestion-job-2021-03-14-14-45-14-849', 'Environment': {'dataset_format': '{"csv

### Explore the generated constraints and statistics

In [14]:
s3_client = boto3.Session().client('s3')
result = s3_client.list_objects(Bucket=bucket, Prefix=baseline_results_prefix)
report_files = [report_file.get("Key") for report_file in result.get('Contents')]
print("Found Files:")
print("\n ".join(report_files))

Found Files:
tf-2-workflow/baselining/results/constraints.json
 tf-2-workflow/baselining/results/statistics.json


In [15]:
import pandas as pd

baseline_job = my_default_monitor.latest_baselining_job
schema_df = pd.io.json.json_normalize(baseline_job.baseline_statistics().body_dict["features"])
schema_df.head(10)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,inferred_type,numerical_statistics.common.num_present,numerical_statistics.common.num_missing,numerical_statistics.mean,numerical_statistics.sum,numerical_statistics.std_dev,numerical_statistics.min,numerical_statistics.max,numerical_statistics.distribution.kll.buckets,numerical_statistics.distribution.kll.sketch.parameters.c,numerical_statistics.distribution.kll.sketch.parameters.k,numerical_statistics.distribution.kll.sketch.data
0,CRIM,Fractional,404,0,3.745111,1513.02467,9.229291,0.00632,88.9762,"[{'lower_bound': 0.00632, 'upper_bound': 8.903...",0.64,2048.0,"[[1.23247, 0.02177, 4.89822, 0.03961, 3.69311,..."
1,ZN,Fractional,404,0,11.480198,4638.00000,23.738277,0.00000,100.0000,"[{'lower_bound': 0.0, 'upper_bound': 10.0, 'co...",0.64,2048.0,"[[0.0, 82.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
2,INDUS,Fractional,404,0,11.104431,4486.19000,6.802873,0.46000,27.7400,"[{'lower_bound': 0.46, 'upper_bound': 3.187999...",0.64,2048.0,"[[8.14, 2.03, 18.1, 5.19, 18.1, 7.38, 18.1, 19..."
3,CHAS,Fractional,404,0,0.061881,25.00000,0.240940,0.00000,1.0000,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,NOX,Fractional,404,0,0.557356,225.17180,0.117148,0.38500,0.8710,"[{'lower_bound': 0.385, 'upper_bound': 0.4336,...",0.64,2048.0,"[[0.538, 0.415, 0.631, 0.515, 0.713, 0.493, 0...."
5,RM,Fractional,404,0,6.267082,2531.90100,0.708909,3.56100,8.7250,"[{'lower_bound': 3.561, 'upper_bound': 4.0774,...",0.64,2048.0,"[[6.142, 7.61, 4.97, 6.037, 6.376, 5.708, 5.53..."
6,AGE,Fractional,404,0,69.010644,27880.30000,27.906063,2.90000,100.0000,"[{'lower_bound': 2.9, 'upper_bound': 12.61, 'c...",0.64,2048.0,"[[91.7, 15.7, 100.0, 34.5, 88.4, 74.3, 100.0, ..."
7,DIS,Fractional,404,0,3.740271,1511.06940,2.027700,1.12960,10.7103,"[{'lower_bound': 1.1296, 'upper_bound': 2.0876...",0.64,2048.0,"[[3.9769, 6.27, 1.3325, 5.9853, 2.5671, 4.7211..."
8,RAD,Fractional,404,0,9.440594,3814.00000,8.687588,1.00000,24.0000,"[{'lower_bound': 1.0, 'upper_bound': 3.3, 'cou...",0.64,2048.0,"[[4.0, 2.0, 24.0, 5.0, 24.0, 5.0, 24.0, 5.0, 5..."
9,TAX,Fractional,404,0,405.898515,163983.00000,166.168506,188.00000,711.0000,"[{'lower_bound': 188.0, 'upper_bound': 240.3, ...",0.64,2048.0,"[[307.0, 348.0, 666.0, 224.0, 666.0, 287.0, 66..."


In [16]:
constraints_df = pd.io.json.json_normalize(baseline_job.suggested_constraints().body_dict["features"])
constraints_df.head(10)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,name,inferred_type,completeness,num_constraints.is_non_negative
0,CRIM,Fractional,1.0,True
1,ZN,Fractional,1.0,True
2,INDUS,Fractional,1.0,True
3,CHAS,Fractional,1.0,True
4,NOX,Fractional,1.0,True
5,RM,Fractional,1.0,True
6,AGE,Fractional,1.0,True
7,DIS,Fractional,1.0,True
8,RAD,Fractional,1.0,True
9,TAX,Fractional,1.0,True


## 2. Analyzing collected data for data quality issues

### Create a schedule

You can create a model monitoring schedule for the endpoint created earlier. Use the baseline resources (constraints and statistics) to compare against the realtime traffic.

In [17]:
from sagemaker.model_monitor import CronExpressionGenerator
from time import gmtime, strftime

mon_schedule_name = 'DEMO-tf-2-workflow-model-monitor-schedule-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
my_default_monitor.create_monitoring_schedule(
    monitor_schedule_name=mon_schedule_name,
    endpoint_input=predictor.endpoint,
    output_s3_uri=s3_report_path,
    statistics=my_default_monitor.baseline_statistics(),
    constraints=my_default_monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,

)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


### Generating violations artificially

In order to get some result relevant to monitoring analysis, you can try and generate artificially some inferences with feature values causing specific violations, and then invoke the endpoint with this data

Looking at our RM and AGE features:

- RM - average number of rooms per dwelling
- AGE - proportion of owner-occupied units built prior to 1940

Let's simulate a situation where the average number of rooms is 0, and proportion of owner-occupied units built is 1000.

In [18]:
df_with_violations = pd.read_csv("training-dataset-with-header.csv")
df_with_violations["RM"] = 0
df_with_violations["AGE"] = 1000
df_with_violations

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,1.23247,0.0,8.14,0.0,0.5380,0,1000,3.9769,4.0,307.0,21.0,396.90,18.72
1,0.02177,82.5,2.03,0.0,0.4150,0,1000,6.2700,2.0,348.0,14.7,395.38,3.11
2,4.89822,0.0,18.10,0.0,0.6310,0,1000,1.3325,24.0,666.0,20.2,375.52,3.26
3,0.03961,0.0,5.19,0.0,0.5150,0,1000,5.9853,5.0,224.0,20.2,396.90,8.01
4,3.69311,0.0,18.10,0.0,0.7130,0,1000,2.5671,24.0,666.0,20.2,391.43,14.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,0.21977,0.0,6.91,0.0,0.4480,0,1000,6.0877,3.0,233.0,17.9,396.90,16.20
400,0.16211,20.0,6.96,0.0,0.4640,0,1000,4.4290,3.0,223.0,18.6,396.90,6.59
401,0.03466,35.0,6.06,0.0,0.4379,0,1000,6.6407,1.0,304.0,16.9,362.25,7.83
402,2.14918,0.0,19.58,0.0,0.8710,0,1000,1.6232,5.0,403.0,14.7,261.95,15.79


### Start generating some artificial traffic
The cell below starts a thread to send some traffic to the endpoint. Note that you need to stop the kernel to terminate this thread. If there is no traffic, the monitoring jobs are marked as `Failed` since there is no data to process.

In [19]:
from threading import Thread
from time import sleep
import time

def invoke_endpoint():
    for item in df_with_violations.to_numpy():
        result = predictor.predict(item)['predictions'] 
        time.sleep(1)

def invoke_endpoint_forever():
    while True:
        invoke_endpoint()
        
thread = Thread(target = invoke_endpoint_forever)
thread.start()

# Note that you need to stop the kernel to stop the invocations

### Describe and inspect the schedule
Once you describe, observe that the MonitoringScheduleStatus changes to Scheduled.

In [21]:
desc_schedule_result = my_default_monitor.describe_schedule()
print('Schedule status: {}'.format(desc_schedule_result['MonitoringScheduleStatus']))

Schedule status: Scheduled


### List executions
The schedule starts jobs at the previously specified intervals. Here, you list the latest five executions. Note that if you are kicking this off after creating the hourly schedule, you might find the executions empty. You might have to wait until you cross the hour boundary (in UTC) to see executions kick off. The code below has the logic for waiting.

Note: Even for an hourly schedule, Amazon SageMaker has a buffer period of 20 minutes to schedule your execution. You might see your execution start in anywhere from zero to ~20 minutes from the hour boundary. This is expected and done for load balancing in the backend.

In [ ]:
mon_executions = my_default_monitor.list_executions()
print("We created a hourly schedule above and it will kick off executions ON the hour (plus 0 - 20 min buffer.\nWe will have to wait till we hit the hour...")

while len(mon_executions) == 0:
    print("Waiting for the 1st execution to happen...")
    time.sleep(60)
    mon_executions = my_default_monitor.list_executions()    

### Inspect a specific execution (latest execution)
In the previous cell, you picked up the latest completed or failed scheduled execution. Here are the possible terminal states and what each of them mean: 
* Completed - This means the monitoring execution completed and no issues were found in the violations report.
* CompletedWithViolations - This means the execution completed, but constraint violations were detected.
* Failed - The monitoring execution failed, maybe due to client error (perhaps incorrect role premissions) or infrastructure issues. Further examination of FailureReason and ExitMessage is necessary to identify what exactly happened.
* Stopped - job exceeded max runtime or was manually stopped.

In [ ]:
latest_execution = mon_executions[-1] # latest execution's index is -1, second to last is -2 and so on..
time.sleep(60)
latest_execution.wait(logs=False)

print("Latest execution status: {}".format(latest_execution.describe()['ProcessingJobStatus']))
print("Latest execution result: {}".format(latest_execution.describe()['ExitMessage']))

latest_job = latest_execution.describe()
if (latest_job['ProcessingJobStatus'] != 'Completed'):
        print("====STOP==== \n No completed executions to inspect further. Please wait till an execution completes or investigate previously reported failures.")

In [ ]:
report_uri=latest_execution.output.destination
print('Report Uri: {}'.format(report_uri))

### List the generated reports

In [ ]:
from urllib.parse import urlparse
s3uri = urlparse(report_uri)
report_bucket = s3uri.netloc
report_key = s3uri.path.lstrip('/')
print('Report bucket: {}'.format(report_bucket))
print('Report key: {}'.format(report_key))

s3_client = boto3.Session().client('s3')
result = s3_client.list_objects(Bucket=report_bucket, Prefix=report_key)
report_files = [report_file.get("Key") for report_file in result.get('Contents')]
print("Found Report Files:")
print("\n ".join(report_files))

### Violations report

If there are any violations compared to the baseline, they will be listed here.

In [ ]:
violations = my_default_monitor.latest_monitoring_constraint_violations()
pd.set_option('display.max_colwidth', -1)
constraints_df = pd.io.json.json_normalize(violations.body_dict["violations"])
constraints_df.head(10)

## Delete the resources

You can keep your endpoint running to continue capturing data. If you do not plan to collect more data or use this endpoint further, you should delete the endpoint to avoid incurring additional charges. Note that deleting your endpoint does not delete the data that was captured during the model invocations. That data persists in Amazon S3 until you delete it yourself.

But before that, you need to delete the schedule first.

In [ ]:
my_default_monitor.delete_monitoring_schedule()
time.sleep(60) # actually wait for the deletion


Deleting Monitoring Schedule with name: DEMO-tf-2-workflow-model-monitor-schedule-2021-03-14-14-59-35


In [ ]:
predictor.delete_endpoint()